# PDEs and their solutions

General partial differential equation I would like to solve:
$$
G\Big( \vec{x}, \; f(\vec{x}), \;  \partial_{i}f(\vec{x}), \; \partial_{i}\partial_{j}f(\vec{x}) \Big) = 0
$$
for $\vec{x} \in [0,1]^n$ (i.e. $n$-dimentional box).

In order for this to have a solution, the following must be true (the *Cauchy–Kowalevski theorem*):

1.  We should be able to solve for the highest derivative. That is, there should exist an analytic $F$ so that 
$$
\partial_k\partial_k f(\vec{x})=F\Big(\vec{x}, f(\vec{x}), \partial_i f(\vec{x}), 
\left\{ \partial_i \partial_j f(\vec{x}) \right\}_{ij \neq kk} \Big)
$$

2. We should know:
$$
f(\vec{x})\Big|_{x_{k}=x_{k}^{(0)}} = L_{0}\Big(\left\{ x_i \right\}_{i \neq k}\Big)
\\
\partial_k f(\vec{x})\Big|_{x_{k}=x_{k}^{(0)}} = L_{1}\Big(\left\{ x_i \right\}_{i \neq k}\Big) 
$$

## Usual PDE problems

In general proving that the given conditions give unique well define solution is non-trivial. However, usually, PDEs are given with some *boundary conditions* of the form
$$
H\Big(\vec{x},f(\vec{x}),\partial_i f(\vec{x}) \Big)\Big|_{\vec{x} \in {\bf S} } =0 \;,
$$
with ${\bf S}$ the boundary of the region in which we look for a solution.

## First order PDEs

In this notebook, we will try to solve general first order PDE inside the $n$-dimensional box. We will assume that there are boundary conditions of the form
$$
H\Big(\vec{x},f(\vec{x}),\partial_i f(\vec{x}) \Big)\Big|_{\vec{x} \in {\bf S} } =0 \;, 
$$
because it will be easier to generalize to 2nd order PDEs later

# Example

As we build the code, let's try to solve the first order PDE
$$
\dfrac{\partial f}{\partial x} +k \dfrac{\partial f}{\partial y} = x
$$

Note that for such PDE we need 

1. An analytic function ($F$) such that $\dfrac{\partial f}{\partial x} = F\left(x,y,f, \dfrac{\partial f}{\partial y}  \right)$ or 
$\dfrac{\partial f}{\partial y} = F\left(x,y,f, \dfrac{\partial f}{\partial x}  \right)$  (which obviously holds).

2. $f(x=x_0,y)=h(y)$ or $f(x,y=y_0)=g(x)$.


The solution is 
$$
f(x,y)=\dfrac{1}{2}x^2 + (y-k \ x) \; c
$$

In [1]:
import numpy as np


import matplotlib
matplotlib.use('nbAgg')
import matplotlib.pyplot as plt

This is how a general function looks-like. It depends on various free parameters (${\bf w}$) that can be adjusted in order to solve the PDE.

Example guess solution.
In genearl, you can change ```__call__```, or pass it as argument, For the moment it is fine.

In [2]:
class Model:
    def __init__(self,w0,dim_w,dim_x):
        self.w=w0
        self.dim_w=dim_w
        self.dim_x=dim_x
        
        
        self.dfdx=[0 for _1 in range(self.dim_x)]
        
        
    def __call__(self,x):
        return self.w[0]*x[0]**2 + self.w[1]*(x[1]-self.w[2]*x[0])
    
    def Deriv(self,x,h=1e-5):
        
        f0=0
        f1=0
        for i in range(self.dim_x):
            x[i]+=h
            f1=self(x)
            
            x[i]+=-2*h
            f0=self(x)
            
            self.dfdx[i]=(f1-f0)/(2*h)



In [3]:
guess=Model(w0=[1,1,2],
                dim_w=3,
                dim_x=2
               )

In [4]:
guess([10,33])

113

In [5]:
guess.Deriv([1,1])

In [6]:
guess.dfdx

[0.0, 1.000000000001]

The boundary condition is a list of functions, based on Model class. 

For this example we can choose $f(0,y) = 0.1 \ y$, as it uniquely  determines the solution. That is, the boundary condition can be writtes=n as a matrix of the form
$$
BC = \left(\begin{matrix} f(0,y) - 0.1 \ y & 0 \\0 & 0 \end{matrix}\right) \; ,
$$
so wou would look for solutions that obey $BC=0$.


So, we can do something like this

In [28]:
class BoundaryCondition:
    def __init__(self,model):
        self.model=model
        
        #zero for everything, and non-zero for the condition you want for this example 
        self.BC=[[lambda x:0,lambda x:0] for _ in range(model.dim_x)]
        self.BC[0][0]=lambda x: model([0,x[1]]) - 0.1*x[1]
    
    def __call__(self,x):
        _bc=[]
        x0=x[:]
        x1=x[:]

        for i in range(self.model.dim_x):
            x0[i]=0
            x1[i]=1

            _bc.append([self.BC[i][0](x0),self.BC[i][1](x1)])

        return _bc

In [29]:
BC=BoundaryCondition(guess)

In [36]:
BC([555,2])

[[1.8, 0], [0, 0]]

In [24]:
class DifferentialEquation:
    def __init__(self,model,A,RHS,boundary_condition):
        
        self.model=model
        
        self.A=A
        
        self.RHS=lambda x: RHS(self.model,x)
        
        self.x0=x0
        
        self.boundary_condition=boundary_condition
        
        
        self.grad=[0 for i in  range(self.model.dim_w)]
        
        
        
        
    
    def loss(self,x):
        # loss at x
        Qx=0        
        self.model.Deriv(x)
        RHS=self.RHS(self,x)
        
        for i in range(self.model.dim_fx):
            for j in range(self.model.dim_x):
                Qx+=(self.model.dfdx[i][j]-RHS[i][j])**2
        
        # loss on the boundary
        Q_bound=0
    
        
        Q=Qx/self.model.dim_x+Q_bound/N_bound
        return Q/2.
    
    
    def lossGrad(self,x,h=1e-5):
        
        for dim in range(self.model.dim_w):

            self.model.w[dim]-=h
            Q0=self.loss(x)

            self.model.w[dim]+=2*h
            Q1=self.loss(x) 
            
            self.model.w[dim]-=h

            self.grad[dim]=(Q1-Q0)/(2*h)

    
    
